In [12]:
from os import environ
from pathlib import Path
environ["OPENAI_API_KEY"] = Path("~/.openaiapikey").expanduser().read_text().strip()
from json import loads

In [6]:
from openai import OpenAI
from random import randint

openaiClient = OpenAI()
def gpt_3_5_turbo_completion(query):
    answer = openaiClient.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": query
            }
        ],
        seed = randint(0, 1000000)
    )
    return answer.choices[0].message.content

def gpt_4_turbo_completion(query):
    answer = openaiClient.chat.completions.create(
        model="gpt-4-turbo",
        messages=[
            {
                "role": "system",
                "content": query
            }
        ],
        seed = randint(0, 1000000)
    )
    return answer.choices[0].message.content

def tryRecieveAnswer(query, completionFunction, answerConversion = lambda x: True, maxTries = 10):
    tryNumber = 0
    while tryNumber < maxTries:
        answer = completionFunction(query)
        try:
            answer = answerConversion(answer)
            return (answer, True)
        except:
            pass
        tryNumber += 1
    print(f"Failed to recieve answer for query: {query}")
    return (None, False)

In [8]:

def getSemanticTriples(subjectTerm):
    querry = f'Semantic triples such as ["Star", "emits", "Light"] and ["Rocket", "can bring cargo to", "Space"] consists of a subject, a predicate, and an object. Give me five examples of semantic triples that contain "{subjectTerm}" as subject and return them in an array formatted like [["sub1", "pred1", "obj1"], ["sub2", "pred2", "obj2"], ...]. Return nothing but the array without explanation.'
    def answerConversion(answer):
        result = loads(answer)
        assert isinstance(result, list)
        assert all(isinstance(triple, list) for triple in result)
        assert all(len(triple) == 3 for triple in result)
        assert all(isinstance(term, str) for triple in result for term in triple)
        return result
    answer, success = tryRecieveAnswer(querry, gpt_4_turbo_completion, answerConversion)
    if success:
        return answer
    return None

In [9]:
getSemanticTriples("Star")

[['Star', 'is a type of', 'Celestial body'],
 ['Star', 'generates', 'Heat'],
 ['Star', 'can be observed in', 'Night sky'],
 ['Star', 'forms from', 'Nebula'],
 ['Star', 'has a lifespan of', 'Millions to billions of years']]

In [21]:
from random import choice
def freeAssociationRandomWalk(startTerm, maxSteps = 10):
    currentSubj = startTerm
    currentPred = None
    currentObj = None
    triplePath = []
    for _ in range(maxSteps):
        querry = f'Semantic triples such as ["Star", "emits", "Light"] and ["Rocket", "can bring cargo to", "Space"] consists of a subject, a predicate, and an object. Give me one example of semantic triples that contains "{currentSubj}" as subject and return it formatted like ["subj", "pred", "obj"]. Return nothing but the triple without explanation.'
        def answerConversion(answer):
            result = loads(answer)
            assert isinstance(result, list)
            assert len(result) == 3
            assert all(isinstance(term, str) for term in result)
            return result
        answer, success = tryRecieveAnswer(querry, gpt_3_5_turbo_completion, answerConversion)
        if success:
            currentSubj, currentPred, currentObj = answer
            print(f"{currentSubj} {currentPred} {currentObj}")
            triplePath.append((currentSubj, currentPred, currentObj))
            currentSubj = currentObj
        else:
            break
    return triplePath

In [25]:
result = freeAssociationRandomWalk("Star")

Star emits Light
Light illuminates Room
Room has Furniture
Furniture enhances interior design
Interior design involves creating functional and aesthetically pleasing spaces
Creating functional and aesthetically pleasing spaces requires attention to detail and creativity
Attention to detail and creativity is important for success
Success brings happiness
Happiness brings joy
Joy brings Happiness


In [42]:
from random import choice
def freeAssociationRandomWalkWithTopic(startTerm, topic, maxSteps = 10):
    currentSubj = startTerm
    currentPred = None
    currentObj = None
    triplePath = []
    for _ in range(maxSteps):
        querry = f'Semantic triples such as ["Star", "emits", "Light"] and ["Rocket", "can bring cargo to", "Space"] consists of a subject, a predicate, and an object. Give me one example of semantic triples that contains "{currentSubj}" as subject. An idea for the topic of the triple is "{topic}". Return the triple formatted like ["subj", "pred", "obj"]. Return nothing but the triple without explanation.'
        def answerConversion(answer):
            result = loads(answer)
            assert isinstance(result, list)
            assert len(result) == 3
            assert all(isinstance(term, str) for term in result)
            return result
        answer, success = tryRecieveAnswer(querry, gpt_4_turbo_completion, answerConversion)
        if success:
            _, currentPred, currentObj = answer
            print(f"{currentSubj} {currentPred} {currentObj}")
            triplePath.append((currentSubj, currentPred, currentObj))
            currentSubj = currentObj
        else:
            break
    return triplePath

In [26]:
result2 = freeAssociationRandomWalkWithTopic("Star", "Particle accelerator")

Star produces Neutrinos
Neutrinos are produced in Particle accelerator
Particle accelerator accelerates Particles
Particles are accelerated using electric fields
Electric fields accelerate Charged particles
Charged particles accelerate Near the speed of light
Near the speed of light propels particles
Particles are accelerated in Particle accelerator
Particle accelerator accelerates Particles
Particles are accelerated by Particle accelerator


In [43]:
result3 = freeAssociationRandomWalkWithTopic("Star", "Harmonic oscillator")

Star oscillates in brightness
brightness is measured in Candelas
Candelas measures Luminous intensity
Luminous intensity is measured in Candelas
Candelas measure luminous intensity of Harmonic oscillator
Harmonic oscillator exhibits simple harmonic motion
simple harmonic motion is exhibited by Harmonic oscillator
Harmonic oscillator exhibits simple harmonic motion
simple harmonic motion is exhibited by Harmonic oscillator
Harmonic oscillator exhibits periodic motion


In [39]:
result3

[('Star', 'exhibits dynamics analogous to', 'Harmonic oscillator'),
 ('Harmonic oscillator', 'undergoes', 'simple harmonic motion'),
 ('simple harmonic motion', 'is a type of motion of', 'Harmonic oscillator'),
 ('Harmonic oscillator', 'exhibits', 'simple harmonic motion'),
 ('simple harmonic motion', 'is exhibited by', 'Harmonic oscillator'),
 ('Harmonic oscillator', 'exhibits', 'simple harmonic motion'),
 ('simple harmonic motion', 'is exhibited by', 'Harmonic oscillator'),
 ('Harmonic oscillator', 'exhibits', 'simple harmonic motion'),
 ('simple harmonic motion', 'is exhibited by', 'Harmonic oscillator'),
 ('Harmonic oscillator', 'undergoes', 'Simple harmonic motion')]

In [ ]:
def createTriplesThatAreLinkingTopics(topics, tripleCount = 5):
    triples = []
    
    return triples

In [44]:
def navigateTriples(triples, startTerm = None):
    while True:
        

hallohallo


In [50]:
from IPython.core.display import display, HTML
display(HTML('<h1>Hello, world!</h1><script>alert("Hello, world!")</script>'))

/tmp/ipykernel_15687/4154482699.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [4]:
from IPython.display import display, HTML
#print((Path("") / "html" / "vis-network.min.js").read_text()[0:200])
display(HTML(
    """
        <!DOCTYPE html>
        <html lang="en-US">
        
        <head>
            <meta charset="UTF-8">
            <meta name="viewport" content="width=device-width, initial-scale=1">
            <title>Inline vis</title>
        </head>
        
        <body>
            <h1 id="testest">Inline vis</h1>
            <div id="mynetwork" style="width:5px; height:5px;"></div>
            <script src=" https://cdn.jsdelivr.net/npm/vis-data@7.1.9/peer/umd/vis-data.min.js "></script>
            <script src=" https://cdn.jsdelivr.net/npm/vis-network@9.1.9/peer/umd/vis-network.min.js "></script>
            <link href=" https://cdn.jsdelivr.net/npm/vis-network@9.1.9/styles/vis-network.min.css " rel="stylesheet">
            <script>

                // create an array with nodes
        document.getElementById("testest").innerText = Object.keys(this).join(", ");
        Network = vis.Network;
        DataSet = vis.DataSet;
        console.log(Network);
        console.log(DataSet);
        // create an array with nodes
        var nodes = new DataSet([
        { id: 1, shape: "dot", size: 20, mass: 10},
        { id: 2, shape: "dot", size: 20, mass: 1},
        { id: 3, shape: "dot", size: 20, mass: 1},
        
        { id: 4, shape: "dot", size: 5, mass: 0.1},
        { id: 5, shape: "dot", size: 5, mass: 0.1},
        { id: 6, shape: "dot", size: 5, mass: 0.1},
        { id: 7, shape: "dot", size: 5, mass: 0.1},
        { id: 8, shape: "dot", size: 5, mass: 0.1},
        ]);

        // create an array with edges
        var edges = new DataSet([
        { from: 1, to: 2, length: 70 },
        { from: 2, to: 3, length: 70 },
        
        { from: 3, to: 4},
        { from: 3, to: 5 },
        { from: 3, to: 6 },
        { from: 3, to: 7 },
        { from: 3, to: 8 },
        ]);

        // create a network
        var data = {
        nodes: nodes,
        edges: edges,
        };
        var container = document.getElementById("mynetwork");
        var options = {};
        var network = new Network(container, data, options);
            </script>
        </body>
        
        </html>

    """))

In [111]:
from IPython.display import display, HTML
print((Path("") / "html" / "vis-network.min.js").read_text()[0:200])
display(HTML(
    """
            <h1 id="testest">Inline vis</h1>
            <div id="mynetwork" style="width:5px; height:5px;"></div>
            <script src=" https://cdn.jsdelivr.net/npm/vis-data@7.1.9/peer/umd/vis-data.min.js "></script>
            <script src=" https://cdn.jsdelivr.net/npm/vis-network@9.1.9/peer/umd/vis-network.min.js "></script>
            <link href=" https://cdn.jsdelivr.net/npm/vis-network@9.1.9/styles/vis-network.min.css " rel="stylesheet">
            <script>

                // create an array with nodes
        document.getElementById("testest").innerText = Object.keys(vis).join(", ");
        Network = vis.Network;
        DataSet = vis.DataSet;
        console.log(Network);
        console.log(DataSet);
        // create an array with nodes
        var nodes = new DataSet([
        { id: 1, shape: "dot", size: 20, mass: 10},
        { id: 2, shape: "dot", size: 20, mass: 1},
        { id: 3, shape: "dot", size: 20, mass: 1},
        
        { id: 4, shape: "dot", size: 5, mass: 0.1},
        { id: 5, shape: "dot", size: 5, mass: 0.1},
        { id: 6, shape: "dot", size: 5, mass: 0.1},
        { id: 7, shape: "dot", size: 5, mass: 0.1},
        { id: 8, shape: "dot", size: 5, mass: 0.1},
        ]);

        // create an array with edges
        var edges = new DataSet([
        { from: 1, to: 2, length: 70 },
        { from: 2, to: 3, length: 70 },
        
        { from: 3, to: 4},
        { from: 3, to: 5 },
        { from: 3, to: 6 },
        { from: 3, to: 7 },
        { from: 3, to: 8 },
        ]);

        // create a network
        var data = {
        nodes: nodes,
        edges: edges,
        };
        var container = document.getElementById("mynetwork");
        var options = {};
        var network = new Network(container, data, options);
            </script>

    """))

/**
 * vis-network
 * https://visjs.github.io/vis-network/
 *
 * A dynamic, browser-based visualization library.
 *
 * @version 9.1.9
 * @date    2023-11-03T01:44:38.007Z
 *
 * @copyright (c) 2011-201


In [6]:
from json import dumps
def safeVisNetworkJSONToHTMLFile(jsonData, htmlFilePath):
    with open(htmlFilePath, "w") as htmlFile:
        htmlFile.write(
            f"""
            <!DOCTYPE html>
            <html lang="en-US">
            
            <head>
                <meta charset="UTF-8">
                <meta name="viewport" content="width=device-width, initial-scale=1">
                <title>Inline vis</title>
            </head>
            
            <body>
                <div id="mynetwork" style="width:100vw; height:100vh;"></div>
                <script src=" https://cdn.jsdelivr.net/npm/vis-data@7.1.9/peer/umd/vis-data.min.js "></script>
                <script src=" https://cdn.jsdelivr.net/npm/vis-network@9.1.9/peer/umd/vis-network.min.js "></script>
                <link href=" https://cdn.jsdelivr.net/npm/vis-network@9.1.9/styles/vis-network.min.css " rel="stylesheet">
                <script>
                    var jsonData = {dumps(jsonData)};
                    // create a network
                    var container = document.getElementById("mynetwork");
                    var options = {{}};
                    var network = new vis.Network(container, jsonData, options);
                </script>
            </body>
            
            </html>
            """
        )

In [5]:
safeVisNetworkJSONToHTMLFile(
    {
        "nodes" : [
            {"id": 1, "label": 'Bla bla bla \nbla bla und so weiter', "group": 'United States', "shape" : "box"},
            {"id": 2, "label": 'Barak Obama', "group": 'United States'},
            {"id": 3, "label": 'Miley Cyrus', "group": 'United States'},
            {"id": 4, "label": 'Pope Francis', "group": 'Vatican'},
            {"id": 5, "label": 'Vladimir Putin', "group": 'Rusia'}
        ],
        "edges" : [
            {"from": 1, "to": 2, "arrows" : "to"},
            {"from": 1, "to": 3},
            {"from": 3, "to": 4},
            {"from": 3, "to": 5}
        ]
    }, "output.html"
)

NameError: name 'safeVisNetworkJSONToHTMLFile' is not defined

In [41]:
def convertSemanticTriplesToVisNetworkJSON(semanticTriples):
    nodes = []
    edges = []
    indexPerTerm = {}
    indexCounter = 1
    for subj, pred, obj in semanticTriples:
        for term in subj, obj:
            if term not in indexPerTerm:
                indexPerTerm[term] = indexCounter
                indexCounter += 1
                nodes.append({
                    "id": indexPerTerm[term], 
                    "label": term, 
                    "font": {"size": 20}, 
                    "color" : {"background": "rgba(140, 220, 255, 255)", 
                                "border": "rgba(0, 0, 255, 255)"}})
        nodes.append({
            "id": indexCounter, 
            "label": pred,
            "font": {"size": 16},
            "color" : {"background": "rgba(160, 255, 100, 255)", 
                        "border": "darkgreen"},
            "shape" : "box"})
        edges.append({"from": indexPerTerm[subj], "to": indexCounter, "color": "rgba(100, 200, 60, 255)", "width": 3})
        edges.append({"from": indexCounter, "to": indexPerTerm[obj], "arrows": "to", "color": "rgba(100, 200, 60, 255)", "width": 3})
        indexCounter += 1
    return {"nodes": nodes, "edges": edges}

In [40]:
triples = [
    ["Star", "emits", "Light"],
    ["Rocket", "can bring cargo to", "Space"],
    ["Star", "shines in", "Space"],
    ["Light", "propagates through", "Vacuum"],
    ["Space", "is mostly a", "Vacuum"]
]
visNetworkJSON = convertSemanticTriplesToVisNetworkJSON(triples)
safeVisNetworkJSONToHTMLFile(visNetworkJSON, "output.html")

In [3]:
def findSemanticTripleObject(subj, pred):
    querry = f'Semantic triples such as ["Star", "emits", "Light"] and ["Rocket", "can bring cargo to", "Space"] consists of a subject, a predicate, and an object. Give me one example of semantic triples that contains "{subj}" as subject and "{pred}" as predicate and return it formatted like ["subj", "pred", "obj"]. Return nothing but the triple without explanation.'
    def answerConversion(answer):
        result = loads(answer)
        assert isinstance(result, list)
        assert len(result) == 3
        assert all(isinstance(term, str) for term in result)
        return result
    answer, success = tryRecieveAnswer(querry, gpt_4_turbo_completion, answerConversion)
    if success:
        return answer
    return None

In [13]:
findSemanticTripleObject("Elementary particle", "has a negatively charged instance")

['Elementary particle', 'has a negatively charged instance', 'Electron']